In [6]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import dart_fss as dart
import json
import pymysql
import info

In [7]:
conn = pymysql.connect(host=info.myHost,user=info.myUser,password=info.myPw,db=info.myDb,charset="utf8")
cur = conn.cursor()

In [8]:
#삼성전자, LG화학, SK하이닉스
company_code_list=["00126380","00356361","00164779"]
bsns_year=["2015","2016","2017","2018","2019","2020"]
#1분기보고서 : 11013, 반기보고서 : 110123, 
#분기보고서 : 11014, 사업보고서 : 11011
reprt_code=["11013","11012","11014","11011"]

In [9]:
sql ="truncate corp_financial_statement"
cur.execute(sql)

for year in bsns_year:
    for reprt in reprt_code:
        for company_code in company_code_list:
            url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key="+info.crtfc_key+"&corp_code="+company_code+"&bsns_year="+year+"&reprt_code="+reprt
            resultXML=urlopen(url)  #this is for response of XML
            stringText=resultXML.read().decode("utf-8")
            data = json.loads(stringText)
        
            #데이터가 접근가능상태인가?
            if(data["status"]=="000"):
                for item in data['list']:
                    #연결재무재표만 query
                    if item['fs_div'] in ['CFS']:   
                        if item['account_nm'] in ['매출액']:
                            ss = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['영업이익']:
                            oi= item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['당기순이익']:
                            ni = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['자본총계']:
                            tl = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['부채총계']:
                            te = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['유동부채']:
                            cl = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['유동자산']:
                            ca = item['thstrm_amount'].replace(",","")
                        if item['account_nm'] in ['비유동자산']:
                            nca = item['thstrm_amount'].replace(",","")
                
                #null check
                if ss:
                    if oi:
                        if ni:
                            if tl:
                                if te:
                                    if cl:
                                        if ca:
                                            if nca:
                                                sql = "INSERT INTO corp_financial_statement values ('"+company_code+"',"+year+",'"+reprt+"',"+ss+","+oi+","+ni+","+tl+","+te+","+ca+","+nca+","+cl+")"
                                                ss, oi, ni, tl, te, cl, ca, nca = None,None,None,None,None,None,None,None
                                                print(sql)
                                                cur.execute(sql)  
                    

INSERT INTO corp_financial_statement values ('00126380',2015,'11011',200653482000000,26413442000000,19060144000000,179059805000000,63119716000000,124814725000000,117364796000000,50502909000000)
INSERT INTO corp_financial_statement values ('00356361',2015,'11011',20206583000000,1823568000000,1148531000000,13103522000000,5475206000000,8655605000000,9923123000000,4798981000000)
INSERT INTO corp_financial_statement values ('00164779',2015,'11011',18797998000000,5336100000000,4323595000000,21387703000000,8290203000000,9760030000000,19917876000000,4840698000000)
INSERT INTO corp_financial_statement values ('00126380',2016,'11013',49782252000000,6675812000000,5252803000000,178326186000000,62913603000000,126587169000000,114652620000000,49049500000000)
INSERT INTO corp_financial_statement values ('00356361',2016,'11013',4874092000000,457745000000,338125000000,13106349000000,5753936000000,8945077000000,9915208000000,5238754000000)
INSERT INTO corp_financial_statement values ('00164779',2016,'110

INSERT INTO corp_financial_statement values ('00356361',2019,'11012',7177444000000,267515000000,83823000000,17278855000000,15156692000000,13301187000000,19134360000000,7362229000000)
INSERT INTO corp_financial_statement values ('00164779',2019,'11012',6452201000000,637630000000,537025000000,47710787000000,15321477000000,14242750000000,48789514000000,7512771000000)
INSERT INTO corp_financial_statement values ('00126380',2019,'11014',62003471000000,7777892000000,6287664000000,263442244000000,89943741000000,186042134000000,167343851000000,63303192000000)
INSERT INTO corp_financial_statement values ('00356361',2019,'11014',7347303000000,380274000000,137185000000,17490783000000,16039976000000,12860159000000,20670600000000,8010251000000)
INSERT INTO corp_financial_statement values ('00164779',2019,'11014',6838766000000,472561000000,495484000000,48266226000000,15405525000000,14175499000000,49496252000000,6827424000000)
INSERT INTO corp_financial_statement values ('00126380',2019,'11011',23040

In [10]:
conn.commit()
conn.close()